<a href="https://colab.research.google.com/github/DanielRaiicHu/TelecomX/blob/main/TelecomX.ipynb?forceRefresh=true">
  <img src="https://drive.google.com/uc?export=view&id=191Woxwut0iaIL2hCcCIeXRSB6oba_88H" alt="Abrir en Colab" width="120"/>
</a>